## Pipeline naive
TODO tell what we do here

We used the (data fusion project) to label objects in the point cloud with the classes: !!TODO!!. It is unknown to which classes the remaining points belong. Therefore, we use the M3C2 algorithm in CloudCompare to remove non-static objects from the point cloud data.


In [1]:
import laspy # TODO also imported in classes
import numpy as np # TODO also imported in classes
import glob
import os

import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.ahn_utils as ahn_utils

from las_dingen import *

## Filter all the points in the point cloud above sidewalk


In [2]:
cloud_run1 = '../datasets/pointclouds/run1/'
cloud_run2 = '../datasets/pointclouds/run2/'

max_obstacle_height = 2.0 # In meters
label_ground = 1
label_obstacle = 15

exclude_label = [1, 2, 7] # We need to ignore ground, buildings and cars

# Create reader for BGT sidewalk part polygons.
bgt_road_file = '../datasets/bgt/bgt_voetpad_demo.csv'
bgt_sidewalk_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_road_file)

# We need elevation data to determine object height above ground.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

In [3]:
# TODO move

from sidewalk_filter import *

def things(points, labels, bgt_sidewalk_reader, tilecode):
    # TODO maak mooier
    sidewalk_polygons = bgt_sidewalk_reader.filter_tile(
                            tilecode, bgt_types=['voetpad'],
                            padding=0, offset=0,
                            merge=False)
    
    # TODO check if bgt_voetpad_demo is empty!
    
    mask = create_mask(labels, exclude_label)

    # Mask obstacles
    label_mask = get_label_mask(points, mask, tilecode, sidewalk_polygons, ahn_reader, max_obstacle_height)

    return label_mask

In [4]:
# Iterate over laz files in folder
for path in glob.glob(f'{cloud_run1}*.laz'):
    filename = os.path.basename(path)
    
    # Start with reading laz file from the other folder
    for pc in [cloud_run2, cloud_run1]:
        # Read
        try: # TODO of os.path.exists
            points, labels = read_las(os.path.join(pc, filename))
        except FileNotFoundError:
            print(f'File {filename} is not present in the other folder. Next...') # TODO better text
            break

        # Check if pc contains ground points
        if label_ground not in labels:
            print(f"No ground/sidewalk points available for file {filename}. Next...")
            break

        # Filter all the points above sidewalk
        tilecode = os.path.splitext(filename)[0].split('processed_')[1]
        label_mask = things(points, labels, bgt_sidewalk_reader, tilecode)

        # Write laz files (TODO make sidewalk folder if not exists)
        write_las(points[label_mask], labels[label_mask], os.path.join(pc, 'sidewalk', filename))

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Saving LAS lidar data
Saving LAS lidar data


## Perform change detection using M3C2 algorithm

In [5]:
# TODO

## Obstacle detection on sidewalks

In [6]:
from accessibility import Accessibility

In [9]:
bgt_data_file = '../datasets/bgt/bgt_voetpad_demo.csv' # TODO

# Read M3C2 result point cloud
points, m3c2, labels = read_las_m3c2('../datasets/pointclouds/m3c2/M3C2_2386_9702.laz') # TODO

# Filter for static objects
mask = ((m3c2 > -0.2) & (m3c2 < 0.2))

# 3D Obstacle blobs to 2D polygons using a clustering algorithm and BGT road data.
accesibility_class = Accessibility(min_component_size=100, bgt_file=bgt_data_file, grid_size=0.05)

label_mask = accesibility_class.get_obstacle_polygons(points, mask, tilecode)

write_las(points[label_mask], labels[label_mask], '../datasets/pointclouds/obstacles/obstacles_2386_9702.laz')
# TODO remove label scalar field


Saving LAS lidar data
